# 什么是上下文管理器？
在任何一门编程语言中，文件的输入输出、数据库的连接断开等，都是很常见的资源管理操作。但资源都是有限的，在写程序时，我们必须保证这些资源在使用过后得到释放，不然就容易造成资源泄露，轻者使得系统处理缓慢，重则会使系统崩溃。
## with 语句
- 上下文管理器，能够帮助你自动分配并且释放资源，其中最典型的应用便是 with 语句。
    - 打开文件
    - 另外一个典型的例子，是 Python 中的 threading.lock 类。


In [1]:
import threading
some_lock = threading.Lock()
some_lock.acquire()
try:
    ...
finally:
    some_lock.release()

In [3]:
'''
 而对应的 with 语句，同样非常简洁：
'''
some_lock = threading.Lock()
with some_lock:
    ...

# 上下文管理器的实现

## 基于类的上下文管理器
 - with的实现依赖于__enter__和__exit__这两个方法。
    - 方法“\__enter__()”返回需要被管理的资源
    - 方法“\__exit__()”里通常会存在一些释放、清理资源的操作
 - 方法“\__exit__()”中的参数“**exc_type, exc_val, exc_tb**”，分别表示 exception_type、exception_value 和 traceback。当我们执行含有上下文管理器的 with 语句时，如果有异常抛出，异常的信息就会包含在这三个变量中，传入方法“\__exit__()”。
 

In [4]:

class FileManager:
    def __init__(self, name, mode):
        print('calling __init__ method')
        self.name = name
        self.mode = mode 
        self.file = None
        
    def __enter__(self):
        print('calling __enter__ method')
        self.file = open(self.name, self.mode)
        return self.file


    def __exit__(self, exc_type, exc_val, exc_tb):
        print('calling __exit__ method')
        if self.file:
            self.file.close()
            
with FileManager('test.txt', 'w') as f:
    print('ready to write to file')
    f.write('hello world')

calling __init__ method
calling __enter__ method
ready to write to file
calling __exit__ method


In [5]:
class Foo:
    def __init__(self):
        print('__init__ called')        

    def __enter__(self):
        print('__enter__ called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('__exit__ called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_tb}')
            print('exception handled')
        return True
    
with Foo() as obj:
    raise Exception('exception raised').with_traceback(None)

__init__ called
__enter__ called
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised
exc_traceback: <traceback object at 0x1076b2640>
exception handled


In [ ]:

class DBConnectionManager: 
    def __init__(self, hostname, port): 
        self.hostname = hostname 
        self.port = port 
        self.connection = None
  
    def __enter__(self): 
        self.connection = DBClient(self.hostname, self.port) 
        return self
  
    def __exit__(self, exc_type, exc_val, exc_tb): 
        self.connection.close() 
  
with DBConnectionManager('localhost', '8080') as db_client:

## 基于生成器的上下文管理器

- 装饰器contextlib.contextmanager

In [ ]:
from contextlib import contextmanager

@contextmanager
def file_manager(name, mode):
    try:
        f = open(name, mode)
        yield f
    finally:
        f.close()
        
with file_manager('test.txt', 'w') as f:
    f.write('hello world')

- 基于类的上下文管理器更加 flexible，适用于大型的系统开发；
- 而基于生成器的上下文管理器更加方便、简洁，适用于中小型程序。